In [1]:
import numpy as np
import math

In [132]:
class MinHeap:
    def __init__(self):
        self.nodes = []

    def get_min(self):
        if len(self.nodes) == 0:
            return -1
        return self.nodes[0]

    def pop_min(self):
        if len(self.nodes) == 0:
            return -1
        elif len(self.nodes) == 1:
            m = self.nodes[0]
            self.nodes = []
            return m
        else:
            m = self.nodes[0]
            e = self.nodes[-1]
            self.nodes = self.nodes[:-1]
            self.nodes[0] = e
            # perculate down
            p = 0 # parent
            l = 2*p + 1 # left
            r = 2*p + 2 # right
            while r < len(self.nodes) and self.nodes[p][0] > min(self.nodes[l][0], self.nodes[r][0]):
                t = self.nodes[p]
                if self.nodes[l][0] > self.nodes[r][0]:
                    self.nodes[p] = self.nodes[r]
                    self.nodes[r] = t
                    p = r
                else:
                    self.nodes[p] = self.nodes[l]
                    self.nodes[l] = t
                    p = l
                l = 2*p + 1
                r = 2*p + 2
            # accounting for odd case where there is not a left and right
            if len(self.nodes) % 2 == 0 and self.nodes[-1][0] < self.nodes[(len(self.nodes) - 2) // 2][0]:
                # swap values
                t = self.nodes[-1]
                self.nodes[-1] = self.nodes[(len(self.nodes) - 2) // 2]
                self.nodes[(len(self.nodes) - 2) // 2] = t
            return m

    def insert(self, key, payload):
        self.nodes.append((key, payload))
        i = len(self.nodes)-1
        p = (i - 1) // 2
        while i > 0 and self.nodes[p][0] > self.nodes[i][0]:
            t = self.nodes[p]
            self.nodes[p] = self.nodes[i]
            self.nodes[i] = t
            i = p
            p = (i - 1) // 2


    def pop_range(self, lteq):
        # gt must be the same data structure as a key.
        results = []
        while len(self.nodes) != 0 and lteq >= self.nodes[0][0]:
            results.append(self.pop_min())
        return results

In [137]:
mh = MinHeap()

n = np.random.randint(1000,size=50)
p = np.random.randint(1000,size=50)
for i in range(50):
    mh.insert(int(n[i]), (int(n[i]),int(p[i])))

In [140]:
print(mh.nodes)

[(86, (86, 665)), (100, (100, 256)), (87, (87, 533)), (166, (166, 879)), (145, (145, 143)), (326, (326, 556)), (451, (451, 275)), (290, (290, 182)), (188, (188, 508)), (163, (163, 821)), (662, (662, 954)), (416, (416, 174)), (444, (444, 300)), (714, (714, 131)), (706, (706, 440)), (670, (670, 942)), (324, (324, 975)), (229, (229, 732)), (206, (206, 717)), (167, (167, 409)), (431, (431, 690)), (668, (668, 505)), (681, (681, 626)), (717, (717, 925)), (458, (458, 750)), (915, (915, 784)), (575, (575, 270)), (810, (810, 380)), (739, (739, 240)), (792, (792, 777)), (728, (728, 856)), (755, (755, 617)), (738, (738, 208)), (723, (723, 624)), (657, (657, 566)), (832, (832, 583)), (606, (606, 51)), (500, (500, 526)), (391, (391, 230)), (687, (687, 138)), (231, (231, 123)), (800, (800, 503)), (465, (465, 724)), (901, (901, 324)), (881, (881, 703)), (806, (806, 981)), (740, (740, 342)), (878, (878, 958))]


In [139]:
mh.pop_range(80)

[(53, (53, 48)), (79, (79, 407))]

In [127]:
sorted(mh.nodes) == [mh.pop_min() for i in range(50)]

True

In [141]:
class PairingHeap:
    def __init__(self, key, payload):
        self.key = key
        self.payload = payload
        self.child = None
        self.left = None
        self.right = None
    
    def meld(self, other):
        """
        Meld other and self. You should only meld 2 roots.
        Parameters:
            other(PairingHeap): PairingHeap to be melded with self
        Return:
            (PairingHeap): resultant pairing heap
        """
        if self.key >= other.key:
            # set other as left child of self
            if self.child is not None:
                self.child.left = other
            other.right = self.child
            self.child = other
            # left of the left-most sibling is the parent
            other.left = self
            return self
        else:
            # set self as left child of other
            if other.child is not None:
                other.child.left = self
            self.right = other.child
            other.child = self
            self.left = other
            return other

    def insert(self, key, payload):
        # probably will not be used if you want to 
        new_tree = PairingHeap(key, payload)
        return self.meld(new_tree)

    def increase_key(self, n, i):
        n.key += i
        # remove from sibling linked list
        if n.right is not None:
            n.right.left = n.left
        if n.left is not None:
            if n.left.child == n:
                n.left.child = n.right
            else:
                n.left.right = n.right
        # meld self with the node with increased key
        return self.meld(n)

    def remove_max(self):
        
        result = (self.key, self.payload)

        # first pass
        children = [self.child]
        while children[-1] is not None:
            if children[-1].right is not None:
                s = children[-1].right
                t = children[-1].right.right
                s.right = None
                s.left = None
                children[-1].right = None
                children[-1].left = None
                children[-1] = children[-1].meld(s)
                children.append(t)
            else:
                children.append(None)
                break
        # remove None from end of list
        children = children[:-1]

        # second pass
        while len(children) > 1:
            children = children[:-2] + [children[-1].meld(children[-2])]
        # If the node is the last in the PairingHeap, then children will have length 0
        # return None as the new PairingHeap
        children.append(None)
        return result, children[0]

    # arbitrary remove
    def arbitrary_remove(self, tree):
        if tree == self: # removing root
            return self.remove_max()
        
        # remove from double linked list
        if tree.left is not None:
            # if left most child
            if tree.left.child == tree:
                tree.left.child = tree.right
            else:
                tree.left.right = tree.right
        if tree.right is not None:
            tree.right.left = tree.left
        
        if tree.child is not None:
            r, m = tree.remove_max()
            return self.meld(m)
        else:
            return self

    def dfs(self):
        # print key in depth first manner
        if self.child is not None:
            self.child.dfs()
        print(self.key)
        if self.right is not None:
            self.right.dfs()

    def __str__(self):
        return f"{self.key}"

In [142]:
np.random.seed(1)
n = np.random.randint(100,size=5)
ph = PairingHeap(n[0],1)
nodes = [ph]
for i in range(1,5):
    nodes.append(PairingHeap(n[i],1))
    ph = ph.meld(nodes[-1])

In [143]:
[n.__str__() for n in nodes]

['37', '12', '72', '9', '75']

In [144]:
ph.dfs()

9
12
37
72
75


In [145]:
ph = ph.arbitrary_remove(nodes[2])

In [146]:
ph.dfs()

9
12
37
75


In [156]:
nodes[-1].child.left.key

np.int64(75)

In [ ]:
class Flight:
    def __init__(self, flightID, airlineID, submitTime, priority, duration):
        self.flightID = flightID
        self.airlineID = airlineID
        self.submitTime = submitTime
        self.priority = 0
        self.duration = 0
        self.statue = "pending"

pending_flights = None # PairingHeap
runway_pool = MinHeap()
active_flights = {} # flightID: Flight
time_table = MinHeap() # ((eta, flightID), (runwayID))
airline_index = {}#airlineID: [flightID]
handles = {}#flightID: PairingHeap

In [ ]:
class Scheduler:
    def __init__(self, n):
        self.pending_flights = None # PairingHeap
        self.runway_pool = MinHeap()
        self.active_flights = {} # flightID: Flight
        self.time_table = MinHeap() # ((eta, flightID), (runwayID))
        self.airline_index = {}#airlineID: [flightID]
        self.handles = {}#flightID: PairingHeap
        
        if n <= 0:
            print("Invalid input. Please provide a valid number of runways.")
        else:
            for i in range(1,n+1):
                self.runway_pool.insert((0,i), (i,0))
            self.currentTime = 0
            self.numRunways = n
            print(f"{n} runways are now available")

    
    def submitFlight(self, flightID, airlineID, submitTime, priority, duration):
        # TODO: settle
        # TODO: reschedule
        self.currentTime = currentTime
        if flightID in self.handles:
            print("Duplicate flight")
        else:
            # new flight
            f = (flightID, airlineID, submitTime, priority, duration, 0)
            new_flight = PairingHeap((priority, -submitTime, -flightID), f)
            handles[flightID] = new_flight
            if self.pending_flights is not None:
                self.pending_flights.meld(new_flight)
            else:
                self.pending_flights = new_flight
            # TODO: reschedule
            print(f"Flight {flightID} scheduled - ETA: {handles[flightID].payload[5]}")
            # TODO: print updated ETAs

    
    def cancelFlight(self, flightID, currentTime):
        # TODO: settle, reschedule
        self.currentTime = currentTime
        if flightID not in self.handles:
            print("Flight does not exist.")
        elif flightID in self.active_flights:
            print(f"Cannot cancel: Flight {flightID} has already departed.")
        else:
            self.pending_flights.remove(handles[flightID])
            # shouldn't be in active_flights
            self.time_table.arbitrary_remove((handles[flightID].payload.eta, flightID))
            # TODO: inefficient method of removing from airline_index
            self.airline_index[handles[flightID].payload.airlineID] = [c for c in self.airline_index[handles[flightID].payload.airlineID] if c != handles[flightID].payload.flightID]
            handles[flightID] = None
            
            # TODO: reschedule
            print(f"Flight {flightID} has been canceled.")

            # TODO: print updated ETAs


    def reprioritize(self, flightID, currentTime, newPriority):
        # TODO: settle, reschedule
        self.currentTime = currentTime
        if flightID not in self.handles:
            print("Flight does not exist.")
        elif flightID in self.active_flights:
            print(f"Cannot reprioritize: Flight {flightID} has already departed.")
        else:
            # TODO: what if newPriority < oldPriority
            self.pending_flights.increase_key(self.handles[flightID], newPriority)
            # TODO: reschedule
            print(f"Priority of Flight {flightID} has been updated to {newPriority}")
            # TODO: print updated ETAs


    def addRunways(self, count, currentTime):
        # TODO: settle, reschedule
        self.currentTime = currentTime
        if count <= 0:
            print("Invalid input. Please provide a valid number of runways.")
        else:
            for i in range(self.numRunways + 1,self.numRunways + 1 + count):
                self.runway_pool.insert((currentTime,i), (i,currentTime))
            # TODO: reschedule
            print(f"Additional {count} runways are now available")
            # TODO: print updated ETAs

    def groundHold(self, airlineLow, airlineHigh, currentTime):
        # TODO: settle, reschedule
        self.currentTime = currentTime
        if airlineHigh < airlineLow:
            print("Invalid input. Please provide a valid airline range.")
        else:
            # TODO: remove flights from data structures
            # TODO: reschedule
            print(f"GroundHold applied to airlines[{airlineLow},{airlineHigh}].")
            # TODO: print updated ETAs


    def printActive(self):
        i = 0
        for key in sorted(self.active_flights.keys()):
            print(self.active_flights[key])
            i = 1
        if i == 0:
            print("No active flights")

    def printSchedule(self, t1, t2):
        flights = [f for f in self.active_flights.values() if f.state == "SCHEDULED" and f.startTime < self.currentTime and t1 <= f.ETA and t2 >= f.ETA]
        for f in sorted(flights, key=lambda f: (f.ETA, f.flightID)):
            print(f"[{f.fligtID}]")
        if len(flights) == 0:
            print("There are no flights in that time period")


    def tick(self, t):
        self.currentTime = t
        # TODO: settle, reschedule
        # TODO: print updated ETAs

In [113]:
ph = ph.increase_key(nodes[2], 10)

In [114]:
ph.dfs()

75
9
12
37
82


In [115]:
r, ph = ph.remove_max()

In [116]:
r

(np.int64(82), 1)

In [117]:
ph.dfs()

12
37
9
75


In [118]:
l = []
for i in range(5):
    r, ph = ph.remove_max()
    l.append(r)
l

AttributeError: 'NoneType' object has no attribute 'remove_max'

In [56]:
ph

In [119]:
l

[(np.int64(75), 1), (np.int64(37), 1), (np.int64(12), 1), (np.int64(9), 1)]

In [120]:
(1,4) + (3,2)

(1, 4, 3, 2)